# Cartpole

In [ ]:
import numpy as np
import time
import math

from pct.hierarchy import PCTHierarchy
from pct.functions import Subtract
from pct.functions import Sigmoid
from pct.functions import Constant
from pct.functions import Proportional
from pct.functions import WeightedSum
from pct.functions import PassOn

from pct.functions import Integration
from pct.functions import IntegrationDual
from pct.functions import OpenAIGym
from pct.functions import IndexedParameter
from pct.functions import GreaterThan

from pct.nodes import PCTNode
from pct.putils import FunctionsList

In [ ]:
cartpole_hierarchy = PCTHierarchy(levels=5, cols=1, name="cartpoleh", build=False)
cartpole_hierarchy.get_node(0, 0).name = 'cart_velocity'
cartpole_hierarchy.get_node(1, 0).name = 'cart_position'
cartpole_hierarchy.get_node(2, 0).name = 'pole_velocity'
cartpole_hierarchy.get_node(3, 0).name = 'pole_angle'
cartpole_hierarchy.get_node(4, 0).name = 'pole_position'

cartpole = OpenAIGym("CartPole-v1", name="CartPole-v1", render=True)
#cartpole = Constant([0.2,0.2,0.2,0.2,0.2+math.sin(0.2)], name="CartPole-v1")
cartpole_hierarchy.set_order("Down")


In [ ]:
# pole_position
FunctionsList.getInstance().get_function("constant4").set_name("pole_position_reference")
FunctionsList.getInstance().get_function("pole_position_reference").set_value(0)
cartpole_hierarchy.insert_function(level=4, col=0, collection="perception", function=IndexedParameter(index=4, name="pole_position", links=[cartpole]))
cartpole_hierarchy.insert_function(level=4, col=0, collection="output", function=Sigmoid(range=0.45, scale=2, name="pole_position_output"))

# pole_angle
cartpole_hierarchy.replace_function(level=3, col=0, collection="reference", function=PassOn(name="pole_angle_reference", links=['pole_position_output']), position=0)
cartpole_hierarchy.insert_function(level=3, col=0, collection="perception", function=IndexedParameter(index=2, name="pole_angle", links=[cartpole]))
FunctionsList.getInstance().get_function("proportional3").set_name("pole_angle_output")
FunctionsList.getInstance().get_function("pole_angle_output").set_property('gain', 1.5)

# pole_velocity
cartpole_hierarchy.insert_function(level=2, col=0, collection="reference", function=PassOn(name="pole_velocity_reference", links=['pole_angle_output']))
cartpole_hierarchy.insert_function(level=2, col=0, collection="perception", function=IndexedParameter(index=3, name="pole_velocity", links=[cartpole]))
cartpole_hierarchy.replace_function(level=2, col=0, collection="output", function=PassOn(name="pole_velocity_output"), position=0)

# cart_position
cartpole_hierarchy.replace_function(level=1, col=0, collection="reference", function=IntegrationDual(name="cart_position_reference", gain=90, slow=100, links=['pole_velocity_output']), position=0)
cartpole_hierarchy.insert_function(level=1, col=0, collection="perception", function=IndexedParameter(index=0, name="cart_position", links=[cartpole]))
FunctionsList.getInstance().get_function("proportional1").set_name("cart_position_output")
FunctionsList.getInstance().get_function("cart_position_output").set_property('gain', 2)

# cart_velocity
cartpole_hierarchy.insert_function(level=0, col=0, collection="reference", function=PassOn(name="cart_velocity_reference", links=['cart_position_output']))
cartpole_hierarchy.insert_function(level=0, col=0, collection="perception", function=IndexedParameter(index=1, name="cart_velocity", links=[cartpole]))
cartpole_hierarchy.insert_function(level=0, col=0, collection="output", function=Integration(gain=-0.1, slow=4, name="force"))


#cartpole_hierarchy.summary()
#cartpole_hierarchy.summary(build=False)


#cartpole_hierarchy.replace_function(level=2, col=0, collection="output", function=Integration(name="pole_velocity_output", gain=10, slow=100), position=0)






In [ ]:


#FunctionsList.getInstance().get_function("proportional2").set_name("pole_velocity_output")
#FunctionsList.getInstance().get_function("pole_velocity_output").set_property('gain', 0.5)



In [ ]:
greaterthan = GreaterThan(threshold=0, upper=1, lower=0, links='force')
cartpole_hierarchy.add_postprocessor(greaterthan)
cartpole_hierarchy.add_preprocessor(cartpole)
cartpole.add_link(greaterthan)

cartpole_hierarchy.summary(build=True)
cartpole_hierarchy.add_links( 'cart_position_reference',  'cart_position')
cartpole_hierarchy.summary(build=False)
#FunctionsList.getInstance().report()


cartpoleh PCTHierarchy
**************************
PRE: CartPole-v1 OpenAIGym | 0 | links  greaterthan 
Level 4
pole_position PCTNode
----------------------------
REF: pole_position_reference Constant | 0 
PER: pole_position IndexedParameter | index 4 | 0 | links  CartPole-v1 
COM: subtract4 Subtract | 0 | links  pole_position_reference pole_position 
OUT: pole_position_output Sigmoid | range 0.45 scale 2  | 0 | links  subtract4 
----------------------------
Level 3
pole_angle PCTNode
----------------------------
REF: pole_angle_reference PassOn | 0 | links  pole_position_output 
PER: pole_angle IndexedParameter | index 2 | 0 | links  CartPole-v1 
COM: subtract3 Subtract | 0 | links  pole_angle_reference pole_angle 
OUT: pole_angle_output Proportional | gain 1.5 | 0 | links  subtract3 
----------------------------
Level 2
pole_velocity PCTNode
----------------------------
REF: pole_velocity_reference PassOn | 0 | links  pole_angle_output 
PER: pole_velocity IndexedParameter | index 3 | 

In [ ]:
#cartpole_hierarchy.draw(font_size=10, figsize=(8,12), move={'CartPole-v1': [-0.075, 0] , 'pole_velocity': [-0.02, 0], 'pole_angle': [-0.025, 0]}, node_size=1000)


In [ ]:
#cartpole_hierarchy.run(10, verbose=True)
cartpole_hierarchy.run(100)

-0.29760973315315303 0
-0.0071547745887334075 0.024225007398387493
0.28741439985670786 0.020386626020779987
0.0025408189142739103 0.020454129973962093
-0.2862656968462607 0.024425863819518157
0.004511209433040982 0.024496953997060415
0.2994746448464319 0.020669124355809018
0.01507154368445859 0.02074716440409054
-0.27318537851874264 0.02472948268043771
0.018215019656638624 0.024811272821106487
0.3138869486220865 0.020994330167726897
0.030285952745373185 0.021083501470084746
-0.25707255147772723 0.02507726564544876
0.03530736055938555 0.025170893265972667
-0.253269808419569 0.021366262285041536
0.037946757648111396 0.021468266414944624
0.3335375096422028 0.017670643195124933
0.04995390084301775 0.01777841062214203
-0.23727872745766784 0.021790154112388202
0.055349177285986484 0.02190124865528502
-0.23287925093506795 0.018113680304301435
0.058801831180081185 0.018232476328040852
0.3549676515405868 0.014451466610971098
0.07207489516913534 0.014575782430658558
-0.21434226822582797 0.018604

0

In [ ]:
cartpole.close()